In [1]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Initialize the memory system
memory_system = MemorySystem()

# Define multiple triples
triples = [
    (URIRef("https://example.org/person/Alice"), URIRef("https://example.org/event/met"), URIRef("https://example.org/person/Bob")), # 0
    (URIRef("https://example.org/person/Bob"), URIRef("https://example.org/event/met"), URIRef("https://example.org/person/Charlie")), # 1
    (URIRef("https://example.org/person/Alice"), URIRef("https://example.org/relationship/knows"), URIRef("https://example.org/person/David")), # 2
    (URIRef("https://example.org/person/Charlie"), URIRef("https://example.org/event/met"), URIRef("https://example.org/person/Eve")), # 3
    (URIRef("https://example.org/person/David"), URIRef("https://example.org/event/invited"), URIRef("https://example.org/person/Alice")), # 4
    (URIRef("https://example.org/person/Eve"), URIRef("https://example.org/relationship/worksWith"), URIRef("https://example.org/person/Bob")), # 5
    (URIRef("https://example.org/person/Charlie"), URIRef("https://example.org/event/attended"), URIRef("https://example.org/person/Bob")), # 6
    (URIRef("https://example.org/entity/Dog"), URIRef("https://example.org/relationship/is"), URIRef("https://example.org/entity/Animal")), # 7
    (URIRef("https://example.org/person/Alice"), URIRef("https://example.org/relationship/owns"), URIRef("https://example.org/entity/Dog")), # 8
    (URIRef("https://example.org/person/Eve"), URIRef("https://example.org/event/met"), URIRef("https://example.org/person/Alice")), # 9
    (URIRef("https://example.org/person/John"), URIRef("https://example.org/relationship/loves"), URIRef("https://example.org/entity/Animal")), # 10
]

# Define qualifiers for episodic memories
episodic_qualifiers_1 = {
    "location": "New York",
    "time": "2024-04-27T15:00:00",
    "emotion": "happy",
    "event": "Coffee meeting"
}
episodic_qualifiers_2 = {
    "location": "London",
    "time": "2024-05-01T10:00:00",
    "emotion": "excited",
    "event": "Conference meeting"
}
episodic_qualifiers_3 = {
    "location": "Paris",
    "time": "2024-05-03T14:00:00",
    "emotion": "curious",
    "event": "Workshop"
}

# Add long-term episodic memories
memory_system.memory.add_long_term_memory("episodic", [triples[0]], **episodic_qualifiers_1)
memory_system.memory.add_long_term_memory("episodic", [triples[1]], **episodic_qualifiers_2)
memory_system.memory.add_long_term_memory("episodic", [triples[2]], **episodic_qualifiers_3)
memory_system.memory.add_long_term_memory("episodic", [triples[3]], **episodic_qualifiers_1)
memory_system.memory.add_long_term_memory("episodic", [triples[4]], **episodic_qualifiers_2)
memory_system.memory.add_long_term_memory("episodic", [triples[5]], **episodic_qualifiers_3)
memory_system.memory.add_long_term_memory("episodic", [triples[6]], **episodic_qualifiers_1)
memory_system.memory.add_long_term_memory("episodic", [triples[7]], **episodic_qualifiers_2)
memory_system.memory.add_long_term_memory("episodic", [triples[8]], **episodic_qualifiers_3)
memory_system.memory.add_long_term_memory("episodic", [triples[9]], **episodic_qualifiers_1)
memory_system.memory.add_long_term_memory("episodic", [triples[10]], **episodic_qualifiers_2)


# Add a semantic memory for the "Dog" entity
semantic_qualifiers = {
    "derivedFrom": "animal_research",
    "strength": 5
}
memory_system.memory.add_long_term_memory("semantic", [triples[7]], **semantic_qualifiers)
memory_system.memory.add_long_term_memory("semantic", [triples[7]], **semantic_qualifiers)
memory_system.memory.add_long_term_memory("semantic", [triples[10]], **semantic_qualifiers)
memory_system.memory.add_long_term_memory("semantic", [triples[10]], **semantic_qualifiers)

# Add a short-term memory (recent event)
memory_system.memory.add_short_term_memory([triples[8]], location="Alice's home")
memory_system.memory.add_short_term_memory([triples[8]], location="Alice's home")

# Add another short-term memory
memory_system.memory.add_short_term_memory([triples[9]], location="Paris Cafe")
memory_system.memory.add_short_term_memory([triples[9]], location="Paris Cafe")

# Function to count triples and memories in working memory
def print_memory_statistics(working_memory, hop_level):
    num_triples = working_memory.get_triple_count()
    num_memories = working_memory.get_memory_count()  # This counts reified statements

    print(f"\n===== Working Memory (Hop {hop_level}) =====")
    print(f"Number of Triples: {num_triples}")
    print(f"Number of Memories (Reified Statements): {num_memories}")
    print(repr(working_memory))

# Test working memory retrieval with different hops
trigger_node = URIRef("https://example.org/person/Alice")

# Hop 0 - Just the immediate node (only the short-term memories involving Alice)
working_memory_hop_0 = memory_system.get_working_memory(trigger_node=trigger_node, hops=0)
print_memory_statistics(working_memory_hop_0, 0)

# Hop 1 - Immediate neighbors (both incoming and outgoing edges)
working_memory_hop_1 = memory_system.get_working_memory(trigger_node=trigger_node, hops=1)
print_memory_statistics(working_memory_hop_1, 1)

# Hop 2 - 2-hop neighbors (further expansion of relationships)
working_memory_hop_2 = memory_system.get_working_memory(trigger_node=trigger_node, hops=2)
print_memory_statistics(working_memory_hop_2, 2)

# Hop 3 - 3-hop neighbors (going even further in the memory graph)
working_memory_hop_3 = memory_system.get_working_memory(trigger_node=trigger_node, hops=3)
print_memory_statistics(working_memory_hop_3, 3)

# Hop 4 - Maximum depth
working_memory_hop_4 = memory_system.get_working_memory(trigger_node=trigger_node, hops=4)
print_memory_statistics(working_memory_hop_4, 4)

# Hop 5 - Maximum depth
working_memory_hop_5 = memory_system.get_working_memory(trigger_node=trigger_node, hops=5)
print_memory_statistics(working_memory_hop_5, 5)

# Example: Get working memory with all long-term memories included
working_memory_all = memory_system.get_working_memory(include_all_long_term=True)
print_memory_statistics(working_memory_all, "All long-term")


===== Working Memory (Hop 0) =====
Number of Triples: 2
Number of Memories (Reified Statements): 4
[Alice, owns, Dog, {'currentTime': '2024-10-03T12:18:27.479883', 'id': '16', 'location': "Alice's home"}]
[Eve, met, Alice, {'id': '18', 'location': 'Paris Cafe', 'currentTime': '2024-10-03T12:18:27.480037'}]
[Alice, owns, Dog, {'location': "Alice's home", 'currentTime': '2024-10-03T12:18:27.479795', 'id': '15'}]
[Eve, met, Alice, {'currentTime': '2024-10-03T12:18:27.479963', 'location': 'Paris Cafe', 'id': '17'}]

===== Working Memory (Hop 1) =====
Number of Triples: 5
Number of Memories (Reified Statements): 9
[Eve, met, Alice, {'currentTime': '2024-10-03T12:18:27.480037', 'location': 'Paris Cafe', 'id': '18'}]
[Alice, owns, Dog, {'currentTime': '2024-10-03T12:18:27.479883', 'location': "Alice's home", 'id': '16'}]
[Eve, met, Alice, {'id': '17', 'location': 'Paris Cafe', 'currentTime': '2024-10-03T12:18:27.479963'}]
[Alice, owns, Dog, {'location': "Alice's home", 'id': '15', 'currentTi

In [2]:
import logging
from rdflib import Graph, URIRef, Namespace, Literal, BNode
from rdflib.namespace import RDF, RDFS, OWL, XSD
from humemai import MemorySystem  # Assuming your module structure

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("humemai.query_example")

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Initialize the memory system
memory_system = MemorySystem()

# Load the ontology into the graph
memory_system.memory.graph.parse("./humemai/humemai.ttl", format="ttl")  # Update the path accordingly

# Assuming you've already added your data as per your previous script
# If not, ensure that the memory data is added before querying

# Bind the namespace for easier query writing
memory_system.memory.graph.bind("humemai", humemai)


In [3]:
short_term_query = """
PREFIX humemai: <https://humem.ai/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?statement ?subject ?predicate ?object ?currentTime ?location
WHERE {
    ?statement rdf:type rdf:Statement ;
               rdf:subject ?subject ;
               rdf:predicate ?predicate ;
               rdf:object ?object ;
               humemai:currentTime ?currentTime .
    
    OPTIONAL { ?statement humemai:location ?location }
}
"""

episodic_query = """
PREFIX humemai: <https://humem.ai/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?statement ?subject ?predicate ?object ?time ?location ?emotion ?event
WHERE {
    ?statement rdf:type rdf:Statement ;
               rdf:subject ?subject ;
               rdf:predicate ?predicate ;
               rdf:object ?object ;
               humemai:time ?time .
    
    OPTIONAL { ?statement humemai:location ?location }
    OPTIONAL { ?statement humemai:emotion ?emotion }
    OPTIONAL { ?statement humemai:event ?event }
}
"""


semantic_query = """
PREFIX humemai: <https://humem.ai/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?statement ?subject ?predicate ?object ?derivedFrom ?strength
WHERE {
    ?statement rdf:type rdf:Statement ;
               rdf:subject ?subject ;
               rdf:predicate ?predicate ;
               rdf:object ?object ;
               humemai:derivedFrom ?derivedFrom .
    
    OPTIONAL { ?statement humemai:strength ?strength }
}
"""
alice_memories_query = """
PREFIX humemai: <https://humem.ai/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?statement ?subject ?predicate ?object ?time ?currentTime ?location ?emotion ?event ?derivedFrom ?strength ?recalled
WHERE {
    ?statement rdf:type rdf:Statement ;
               rdf:subject <https://example.org/person/Alice> ;
               rdf:predicate ?predicate ;
               rdf:object ?object .
    
    OPTIONAL { ?statement humemai:time ?time }
    OPTIONAL { ?statement humemai:currentTime ?currentTime }
    OPTIONAL { ?statement humemai:location ?location }
    OPTIONAL { ?statement humemai:emotion ?emotion }
    OPTIONAL { ?statement humemai:event ?event }
    OPTIONAL { ?statement humemai:derivedFrom ?derivedFrom }
    OPTIONAL { ?statement humemai:strength ?strength }
    OPTIONAL { ?statement humemai:recalled ?recalled }
}
"""



In [4]:
def execute_query(graph, query):
    return graph.query(query)

# Execute Short-Term Memories Query
short_term_results = execute_query(memory_system.memory.graph, short_term_query)
print("===== Short-Term Memories =====")
for row in short_term_results:
    print(f"Memory: {row.memory}")
    print(f"Subject: {row.subject}")
    print(f"Predicate: {row.predicate}")
    print(f"Object: {row.object}")
    print(f"Current Time: {row.currentTime}")
    print(f"Location: {row.location}")
    print("-----")

# Execute Episodic Memories Query
episodic_results = execute_query(memory_system.memory.graph, episodic_query)
print("\n===== Episodic Memories =====")
for row in episodic_results:
    print(f"Memory: {row.memory}")
    print(f"Subject: {row.subject}")
    print(f"Predicate: {row.predicate}")
    print(f"Object: {row.object}")
    print(f"Time: {row.time}")
    print(f"Location: {row.location}")
    print(f"Emotion: {row.emotion}")
    print(f"Event: {row.event}")
    print("-----")

# Execute Semantic Memories Query
semantic_results = execute_query(memory_system.memory.graph, semantic_query)
print("\n===== Semantic Memories =====")
for row in semantic_results:
    print(f"Memory: {row.memory}")
    print(f"Subject: {row.subject}")
    print(f"Predicate: {row.predicate}")
    print(f"Object: {row.object}")
    print(f"Derived From: {row.derivedFrom}")
    print(f"Strength: {row.strength}")
    print("-----")

# Execute Alice's Memories Query
alice_memories_results = execute_query(memory_system.memory.graph, alice_memories_query)
print("\n===== Alice's Memories =====")
for row in alice_memories_results:
    print(f"Memory: {row.memory}")
    print(f"Type: {row.type}")
    print(f"Predicate: {row.predicate}")
    print(f"Object: {row.object}")
    qualifiers = []
    if 'time' in row:
        qualifiers.append(f"Time: {row.time}")
    if 'currentTime' in row:
        qualifiers.append(f"Current Time: {row.currentTime}")
    if 'location' in row:
        qualifiers.append(f"Location: {row.location}")
    if 'emotion' in row:
        qualifiers.append(f"Emotion: {row.emotion}")
    if 'event' in row:
        qualifiers.append(f"Event: {row.event}")
    if 'derivedFrom' in row:
        qualifiers.append(f"Derived From: {row.derivedFrom}")
    if 'strength' in row:
        qualifiers.append(f"Strength: {row.strength}")
    if 'recalled' in row:
        qualifiers.append(f"Recalled: {row.recalled}")
    print(", ".join(qualifiers))
    print("-----")


===== Short-Term Memories =====

===== Episodic Memories =====

===== Semantic Memories =====

===== Alice's Memories =====
